In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import talib as ta
import datetime, time
import pickle
from tqdm import tqdm_notebook

import pdblp as pb
import blpapi 

In [2]:
from utils import *

In [3]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

In [3]:
con = pb.BCon(port=8194, timeout=5000)
con.start()

In [17]:
df_code = pd.read_excel('data/etf_pair_code.xlsx', dtype={'symbol':str})
code_list = df_code.symbol.unique().tolist()
ticker_list = (df_code.symbol + ' ' + df_code.region).fillna('ita').unique().tolist()

### test 1

In [5]:
df = pd.DataFrame()
err_code = []
tic = time.perf_counter()
fields = ['HIGH', 'OPEN', 'LOW', 'VOLUME', 'PX_LAST']
cols = ['High', 'Open', 'Low', 'Volume', 'Adj Close']
start = '20100101'
end = '20190115'



for num,i in tqdm_notebook(enumerate(ticker_list)):
    if (num % 100 == 0) & (num != 0):
        time.sleep(5)
    try:
        temp_df = con.bdh(i, fields, start, end)
        temp_df = temp_df.loc[:, i]
        temp_df.columns = cols
        temp_df = temp_df.assign(code=i.split(' ')[0])

        try:
            df = pd.concat([df, temp_df], axis=0)
        except:
            print('concat data has problem, ticker code : %s' % i)

    except:
        print('retrieve %s data, have something wrong, index:%s' % (i, ticker_list.index(i)))
        err_code.append(i)
toc = time.perf_counter()

print('retrieve data cost time: %s' % (toc-tic))

retrieve IUGXX US Equity data, have something wrong, index:145
retrieve HKD Curncy data, have something wrong, index:210
retrieve TRPXX US Equity data, have something wrong, index:275
retrieve GVMXX US Equity data, have something wrong, index:413
retrieve BRIXX US Equity data, have something wrong, index:511
retrieve ita data, have something wrong, index:620
retrieve KORS US Equity data, have something wrong, index:718

retrieve data cost time: 1024.268109704


In [6]:
df.head()

,High,Open,Low,Volume,Adj Close,code
date,,,,,,
2010-01-04,24.8407,24.4573,24.4334,38414185.0,24.7209,MSFT
2010-01-05,24.8407,24.6410,24.4733,49758862.0,24.7289,MSFT
2010-01-06,24.8247,24.6650,24.3775,58182332.0,24.5771,MSFT
2010-01-07,24.5212,24.4653,24.1139,50564285.0,24.3231,MSFT
2010-01-08,24.6650,24.1858,24.1538,51201289.0,24.4893,MSFT


In [7]:
df.tail()

,High,Open,Low,Volume,Adj Close,code
date,,,,,,
2019-01-09,36.1600,35.92,35.24,1582666.0,35.27,SEE
2019-01-10,36.3200,35.21,35.20,1141760.0,36.09,SEE
2019-01-11,36.7281,36.01,35.81,1425600.0,36.57,SEE
2019-01-14,36.6500,36.38,36.09,1130188.0,36.48,SEE
2019-01-15,36.7000,36.38,36.20,781775.0,36.67,SEE


In [8]:
df.to_csv('data/data.csv')

In [9]:
%load_ext autoreload
%autoreload 2

from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
result = calc_pair_trading('SMH', 
                           data_source='ol',
                           region='all',
                           method='rolling', 
                           period=245, 
                           is_saving=True, 
                           save_name='rsi_sample')


finish, see you          


如果上述代码没有问题，可以尝试以下代码

### test 2

暂先剔除HKD Curncy 和 ita

In [11]:
ticker_list.remove('HKD Curncy')
ticker_list.remove('ita')

In [12]:
intervals = np.linspace(0, len(ticker_list), 10, dtype=int)

In [13]:
tic = time.perf_counter()

df = pd.DataFrame()

for i in tqdm_notebook(range(len(intervals)-1)):
    
    time.sleep(5)
    
    temp_list = ticker_list[intervals[i]:intervals[i+1]]
    
    temp_df = con.bdh(temp_list, fields, start, end)

    temp_df = pd.concat([temp_df[x].assign(code=x.split(' ')[0]).groupby(level=0, axis=1).first() for x in temp_df.columns.levels[0]], 
              axis=0,sort=False).reset_index().set_index(['date', 'code'])
    temp_df.columns = cols
    
    df = pd.concat([df, temp_df], axis=0)

toc = time.perf_counter()

print('retrieve data cost time: %s' % (toc-tic))


retrieve data cost time: 184.01159243400002


In [14]:
# tic = time.perf_counter()

# df = con.bdh(ticker_list, fields, start, end)

# df = pd.concat([df[x].assign(code=x.split(' ')[0]).groupby(level=0, axis=1).first() for x in df.columns.levels[0]], 
#           axis=0,sort=False).reset_index().set_index(['date', 'code'])
# df.columns = cols

# toc = time.perf_counter()

# print('retrieve data cost time: %s' % (toc-tic))

In [15]:
df.to_csv('data/data.csv')

In [12]:
result = calc_pair_trading('SMH', 
                           data_source='ol',
                           region='all',
                           method='rolling', 
                           period=245, 
                           is_saving=False, 
                           save_name='rsi_sample')

In [13]:
print(result['long_return'].sum().sum(), 
     result['long_short_return'].sum().sum())

0.251653963144 0.017070768697


In [11]:
print(result['long_return'].sum().sum(), 
     result['long_short_return'].sum().sum())

0.246968945279 0.0247123504036


In [34]:
print(result['long_return'].sum().sum(), 
     result['long_short_return'].sum().sum())

0.753622105247 0.115923496525


In [35]:
ol_data.head()

,,High,Low,Open,Adj Close,Volume
date,code,,,,,
2010-01-04,AABA,17.20,16.880,16.94,17.10,16588957.0
2010-01-05,AABA,17.23,17.000,17.22,17.23,11718126.0
2010-01-06,AABA,17.30,17.070,17.17,17.17,16421960.0
2010-01-07,AABA,16.90,16.570,16.81,16.70,31816301.0
2010-01-08,AABA,16.76,16.615,16.68,16.70,15471074.0


In [6]:
df.head()

ticker     ADP US Equity                                       ADBE US Equity  \
field               HIGH     OPEN      LOW     VOLUME  PX_LAST           HIGH   
date                                                                            
2010-01-04       30.3130  30.3130  29.7247  3450425.0  29.8187        37.3000   
2010-01-05       29.8744  29.6934  29.4984  2856017.0  29.6585        37.8000   
2010-01-06       29.8604  29.7978  29.2756  2731632.0  29.5889        37.7400   
2010-01-07       29.5889  29.3313  29.3035  2206516.0  29.5750        37.5865   
2010-01-08       29.5611  29.5332  29.2687  2028462.0  29.5332        36.9700   

ticker                                        ...    ORCL US Equity           \
field        OPEN    LOW     VOLUME PX_LAST   ...              HIGH     OPEN   
date                                          ...                              
2010-01-04  36.65  36.65  4710112.0   37.09   ...           22.4846  22.0116   
2010-01-05  37.04  36.87  7108727.0   37.70   ...           22.1812  22.0651   
2010-01-06  37.33  37.20  5336345.0   37.62   ...           22.2392  22.1098   
2010-01-07  37.41  36.81  5576649.0   36.89   ...           21.9669  21.8330   
2010-01-08  36.75  36.34  5429158.0   36.69   ...           22.0919  21.6724   

ticker                                   AVGO US Equity                    \
field           LOW      VOLUME  PX_LAST           HIGH     OPEN      LOW   
date                                                                        
2010-01-04  22.0116  26795074.0  22.1812        16.2960  15.7786  15.5286   
2010-01-05  21.7349  28669970.0  22.1544        16.5891  16.3391  15.6062   
2010-01-06  21.7616  24560772.0  21.8330        16.6409  16.3391  16.0287   
2010-01-07  21.4939  30469731.0  21.7616        16.5288  16.4167  16.0804   
2010-01-08  21.6456  23542432.0  22.0294        16.6409  16.2960  16.2530   

ticker                         
field         VOLUME  PX_LAST  
date                           
2010-01-04  538645.0  16.2615  
2010-01-05  301037.0  16.3822  
2010-01-06  533035.0  16.5115  
2010-01-07  271729.0  16.4081  
2010-01-08  289844.0  16.5288  

[5 rows x 100 columns]

In [16]:
df.columns.get_level_values(0).unique()

Index(['ADP US Equity', 'ADBE US Equity', 'MSFT US Equity', 'INTU US Equity',
       'MU US Equity', 'IBM US Equity', 'PYPL US Equity', 'CRM US Equity',
       'CTSH US Equity', 'ACN US Equity', 'AAPL US Equity', 'TXN US Equity',
       'V US Equity', 'INTC US Equity', 'NVDA US Equity', 'CSCO US Equity',
       'QCOM US Equity', 'MA US Equity', 'ORCL US Equity', 'AVGO US Equity'],
      dtype='object', name='ticker')

In [19]:
ticker_list.remove('HKD Curncy')
ticker_list.remove('ita')
temp_list = ticker_list[:20]
np.array(temp_list)

array(['MSFT US Equity', 'AAPL US Equity', 'V US Equity', 'INTC US Equity',
       'CSCO US Equity', 'MA US Equity', 'ORCL US Equity',
       'AVGO US Equity', 'ADBE US Equity', 'IBM US Equity',
       'PYPL US Equity', 'CRM US Equity', 'ACN US Equity', 'TXN US Equity',
       'NVDA US Equity', 'QCOM US Equity', 'ADP US Equity',
       'INTU US Equity', 'MU US Equity', 'CTSH US Equity'],
      dtype='<U14')

In [6]:
with open('data/temp_dataframe', 'rb') as f:
    temp_df = pickle.load(f)

In [7]:
temp_df.head()

ticker     ADP US Equity                                       ADBE US Equity  \
field               HIGH     OPEN      LOW     VOLUME  PX_LAST           HIGH   
date                                                                            
2010-01-04       30.3130  30.3130  29.7247  3450425.0  29.8187        37.3000   
2010-01-05       29.8744  29.6934  29.4984  2856017.0  29.6585        37.8000   
2010-01-06       29.8604  29.7978  29.2756  2731632.0  29.5889        37.7400   
2010-01-07       29.5889  29.3313  29.3035  2206516.0  29.5750        37.5865   
2010-01-08       29.5611  29.5332  29.2687  2028462.0  29.5332        36.9700   

ticker                                        ...    ORCL US Equity           \
field        OPEN    LOW     VOLUME PX_LAST   ...              HIGH     OPEN   
date                                          ...                              
2010-01-04  36.65  36.65  4710112.0   37.09   ...           22.4846  22.0116   
2010-01-05  37.04  36.87  7108727.0   37.70   ...           22.1812  22.0651   
2010-01-06  37.33  37.20  5336345.0   37.62   ...           22.2392  22.1098   
2010-01-07  37.41  36.81  5576649.0   36.89   ...           21.9669  21.8330   
2010-01-08  36.75  36.34  5429158.0   36.69   ...           22.0919  21.6724   

ticker                                   AVGO US Equity                    \
field           LOW      VOLUME  PX_LAST           HIGH     OPEN      LOW   
date                                                                        
2010-01-04  22.0116  26795074.0  22.1812        16.2960  15.7786  15.5286   
2010-01-05  21.7349  28669970.0  22.1544        16.5891  16.3391  15.6062   
2010-01-06  21.7616  24560772.0  21.8330        16.6409  16.3391  16.0287   
2010-01-07  21.4939  30469731.0  21.7616        16.5288  16.4167  16.0804   
2010-01-08  21.6456  23542432.0  22.0294        16.6409  16.2960  16.2530   

ticker                         
field         VOLUME  PX_LAST  
date                           
2010-01-04  538645.0  16.2615  
2010-01-05  301037.0  16.3822  
2010-01-06  533035.0  16.5115  
2010-01-07  271729.0  16.4081  
2010-01-08  289844.0  16.5288  

[5 rows x 100 columns]

In [8]:
temp_df = pd.concat([temp_df[x].assign(code=x.split(' ')[0]).groupby(level=0, axis=1).first() for x in temp_df.columns.levels[0]], 
              axis=0,sort=False)

In [9]:
temp_df.head()

field,HIGH,LOW,OPEN,PX_LAST,VOLUME,code
date,,,,,,
2010-01-04,27.0630,26.7955,26.9280,27.0011,123432050.0,AAPL
2010-01-05,27.2005,26.9053,27.0756,27.0478,150476004.0,AAPL
2010-01-06,27.1551,26.5898,27.0478,26.6176,138039594.0,AAPL
2010-01-07,26.7476,26.3754,26.7160,26.5684,119269535.0,AAPL
2010-01-08,26.7476,26.3766,26.5331,26.7450,111969081.0,AAPL


In [49]:
temp_df[['HIGH', 'OPEN', 'LOW', 'VOLUME', 'PX_LAST', 'code']]

field,HIGH,OPEN,LOW,VOLUME,PX_LAST,code
date,,,,,,
2010-01-04,27.0630,26.9280,26.7955,123432050.0,27.0011,AAPL
2010-01-05,27.2005,27.0756,26.9053,150476004.0,27.0478,AAPL
2010-01-06,27.1551,27.0478,26.5898,138039594.0,26.6176,AAPL
2010-01-07,26.7476,26.7160,26.3754,119269535.0,26.5684,AAPL
2010-01-08,26.7476,26.5331,26.3766,111969081.0,26.7450,AAPL
2010-01-11,26.8737,26.8485,26.2997,115557365.0,26.5091,AAPL
2010-01-12,26.4662,26.3930,26.0435,148614774.0,26.2076,AAPL
2010-01-13,26.6126,26.2265,25.7508,151472335.0,26.5772,AAPL
2010-01-14,26.5533,26.5091,26.3716,108288411.0,26.4233,AAPL


In [10]:
cols = ['High', 'Open', 'Low', 'Volume', 'Adj Close']
temp_df.columns = cols

In [11]:
temp_df.head()

,,High,Open,Low,Volume,Adj Close
date,code,,,,,
2010-01-04,AAPL,27.0630,26.7955,26.9280,27.0011,123432050.0
2010-01-05,AAPL,27.2005,26.9053,27.0756,27.0478,150476004.0
2010-01-06,AAPL,27.1551,26.5898,27.0478,26.6176,138039594.0
2010-01-07,AAPL,26.7476,26.3754,26.7160,26.5684,119269535.0
2010-01-08,AAPL,26.7476,26.3766,26.5331,26.7450,111969081.0


In [5]:
test = pd.read_csv('data/data.csv')

In [8]:
test[['date', 'High', 'Open', 'Low', 'Volume',
                               'Adj Close', 'code']].to_csv('data/data.csv', index=False)

In [4]:
ol_data.head()

,,high,low,open,close,volume,Adj Close
date,code,,,,,,
0,AABA,2010-01-04,17.20,16.94,16.880,16588957.0,17.10
1,AABA,2010-01-05,17.23,17.22,17.000,11718126.0,17.23
2,AABA,2010-01-06,17.30,17.17,17.070,16421960.0,17.17
3,AABA,2010-01-07,16.90,16.81,16.570,31816301.0,16.70
4,AABA,2010-01-08,16.76,16.68,16.615,15471074.0,16.70


In [41]:
ol_data = ol_data.reset_index()[['date', 'High', 'Open', 'Low', 'Volume',
                               'Adj Close', 'code']]

In [50]:
ol_data.to_csv('data/data.csv')

In [42]:
ol_data.head()

,date,High,Open,Low,Volume,Adj Close,code
0,2010-01-04,17.20,16.94,16.880,16588957.0,17.10,AABA
1,2010-01-05,17.23,17.22,17.000,11718126.0,17.23,AABA
2,2010-01-06,17.30,17.17,17.070,16421960.0,17.17,AABA
3,2010-01-07,16.90,16.81,16.570,31816301.0,16.70,AABA
4,2010-01-08,16.76,16.68,16.615,15471074.0,16.70,AABA


In [39]:
ol_data.index.names

FrozenList(['date', 'code'])

In [36]:
ol_data.columns

Index(['High', 'Low', 'Open', 'Adj Close', 'Volume'], dtype='object')